# Evaluation of the operator constructed in openfermion on the state.


In [64]:
from pyqtorch.core.operation import Z, Y, X, RX
from pyqtorch.embedding import SingleLayerEncoding
from pyqtorch.ansatz import AlternateLayerAnsatz
from pyqtorch.core.circuit import QuantumCircuit
from pyqtorch.core.measurement import total_magnetization

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

import math
import networkx as nx
import numpy as np
import matplotlib as mpl

mpl.rcParams["figure.figsize"] = [5, 5]

In [70]:
from openfermion.hamiltonians import fermi_hubbard
from openfermion.linalg import get_sparse_operator, get_ground_state
from openfermion.transforms import jordan_wigner


# Set model.
x_dimension = 2
y_dimension = 2
tunneling = 2.0
coulomb = 1.0
magnetic_field = 0.5
chemical_potential = 0.25
periodic = 1
spinless = 1

# Get fermion operator.
hubbard_model = fermi_hubbard(
    x_dimension,
    y_dimension,
    tunneling,
    coulomb,
    chemical_potential,
    magnetic_field,
    periodic,
    spinless,
)
# print(hubbard_model)

# Get qubit operator under Jordan-Wigner.
jw_hamiltonian = jordan_wigner(hubbard_model)
# Removes zero entries.
jw_hamiltonian.compress()
print("")
print(jw_hamiltonian)

# Get scipy.sparse.csc representation.
sparse_operator = get_sparse_operator(hubbard_model)
print("")
print(sparse_operator)
print(
    "\nEnergy of the model is {} in units of T and J.".format(
        get_ground_state(sparse_operator)[0]
    )
)


0.5 [] +
-1.0 [X0 X1] +
-1.0 [X0 Z1 X2] +
-1.0 [Y0 Y1] +
-1.0 [Y0 Z1 Y2] +
-0.375 [Z0] +
0.25 [Z0 Z1] +
0.25 [Z0 Z2] +
-1.0 [X1 Z2 X3] +
-1.0 [Y1 Z2 Y3] +
-0.375 [Z1] +
0.25 [Z1 Z3] +
-1.0 [X2 X3] +
-1.0 [Y2 Y3] +
-0.375 [Z2] +
0.25 [Z2 Z3] +
-0.375 [Z3]

  (1, 1)	(-0.25+0j)
  (2, 1)	(-2+0j)
  (4, 1)	(-2+0j)
  (1, 2)	(-2+0j)
  (2, 2)	(-0.25+0j)
  (8, 2)	(-2+0j)
  (3, 3)	(0.5+0j)
  (6, 3)	(2+0j)
  (9, 3)	(-2+0j)
  (1, 4)	(-2+0j)
  (4, 4)	(-0.25+0j)
  (8, 4)	(-2+0j)
  (5, 5)	(0.5+0j)
  (6, 5)	(-2+0j)
  (9, 5)	(-2+0j)
  (3, 6)	(2+0j)
  (5, 6)	(-2+0j)
  (6, 6)	(-0.5+0j)
  (10, 6)	(-2+0j)
  (12, 6)	(2+0j)
  (7, 7)	(1.25+0j)
  (11, 7)	(-2+0j)
  (13, 7)	(2+0j)
  (2, 8)	(-2+0j)
  (4, 8)	(-2+0j)
  (8, 8)	(-0.25+0j)
  (3, 9)	(-2+0j)
  (5, 9)	(-2+0j)
  (9, 9)	(-0.5+0j)
  (10, 9)	(-2+0j)
  (12, 9)	(-2+0j)
  (6, 10)	(-2+0j)
  (9, 10)	(-2+0j)
  (10, 10)	(0.5+0j)
  (7, 11)	(-2+0j)
  (11, 11)	(1.25+0j)
  (14, 11)	(2+0j)
  (6, 12)	(2+0j)
  (9, 12)	(-2+0j)
  (12, 12)	(0.5+0j)
  (7, 13)	(2+0j)
  (13, 13

In [71]:
from openfermion.utils import count_qubits

n_qubits = count_qubits(jw_hamiltonian)

In [72]:
# Generate operators for the hamiltonian.
operator = jw_hamiltonian
for op, coef in operator.terms.items():
    print(op, coef)
    for qubit, pauli in op:
        print(qubit, pauli)

((0, 'Y'), (1, 'Y')) -1.0
0 Y
1 Y
((0, 'X'), (1, 'X')) -1.0
0 X
1 X
() 0.5
((1, 'Z'),) -0.375
1 Z
((0, 'Z'),) -0.375
0 Z
((0, 'Z'), (1, 'Z')) 0.25
0 Z
1 Z
((0, 'Y'), (1, 'Z'), (2, 'Y')) -1.0
0 Y
1 Z
2 Y
((0, 'X'), (1, 'Z'), (2, 'X')) -1.0
0 X
1 Z
2 X
((2, 'Z'),) -0.375
2 Z
((0, 'Z'), (2, 'Z')) 0.25
0 Z
2 Z
((1, 'Y'), (2, 'Z'), (3, 'Y')) -1.0
1 Y
2 Z
3 Y
((1, 'X'), (2, 'Z'), (3, 'X')) -1.0
1 X
2 Z
3 X
((3, 'Z'),) -0.375
3 Z
((1, 'Z'), (3, 'Z')) 0.25
1 Z
3 Z
((2, 'Y'), (3, 'Y')) -1.0
2 Y
3 Y
((2, 'X'), (3, 'X')) -1.0
2 X
3 X
((2, 'Z'), (3, 'Z')) 0.25
2 Z
3 Z


In [73]:
# Generate state with different batches.
batch_size = 3
state = QuantumCircuit(n_qubits).init_state(batch_size)
# Reshaping was only necessary for the expectation value.
# state = state.reshape((2**n_qubits, batch_size))
print(state)
print(state.shape)


tensor([[[[[1.+0.j, 1.+0.j, 1.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j]],

          [[0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j]]],


         [[[0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j]],

          [[0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j]]]],



        [[[[0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j]],

          [[0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j]]],


         [[[0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j]],

          [[0.+0.j, 0.+0.j, 0.+0.j],
           [0.+0.j, 0.+0.j, 0.+0.j]]]]], dtype=torch.complex128)
torch.Size([2, 2, 2, 2, 3])


In [76]:
from pyqtorch.core.measurement import measure_openfermion

measure_openfermion(state, jw_hamiltonian, n_qubits, batch_size)

tensor([-3.5000, -3.5000, -3.5000], dtype=torch.float64)

In [77]:
total_magnetization(state, n_qubits, batch_size)

tensor([4., 4., 4.], dtype=torch.float64)

#### The state is |0000> since every state 0 contributes with spin +1 to the total magnerization.


***
# Outcome:

- We can create any operator in openfermion and then evaluate the expectation.
